# Import Libraries

In [1]:
import pickle
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Importing Data

In [2]:
with open('new_eda_data.pkl', 'rb') as file:
    df = pickle.load(file)

In [3]:
# select columns for modelling
df_tags = df[['tags', 'target']].copy()

In [4]:
df_tags.reset_index(drop=True, inplace=True)

In [5]:
df_tags

,tags,target
0,[],1
1,[],0
2,[],1
3,[],0
4,[],0
...,...,...
8499,"[Data Analyst, How to become a data analyst, D...",1
8500,"[Data Analyst, How to become a Data Analyst, B...",1
8501,"[Data Analyst Salary, Data analyst with no exp...",1
8502,"[Working at a big company, Big company data an...",1


In [25]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8504 entries, 0 to 8503
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tags    8504 non-null   object
 1   target  8504 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 133.0+ KB


In [6]:
# store unique tag count across df
unique_tags = set()

# iterate through unique tags and map their count
for tag_list in df_tags['tags']:

    for tag in tag_list:
       unique_tags.add(tag.lower())

len(unique_tags)

31997

In [ ]:
# create empty columns of all unique tags
for tag in unique_tags:
    df_tags[tag] = df_tags['tags'].apply(lambda x: 1 if tag in x else 0)